# GEBCO

Denne notebooken er en POC for å laste ned data fra Gebco og klargjøre den for nye Topo.

In [ ]:
%pip install numpy pyproj rasterio rio-cogeo shapely

In [93]:
import json
from pathlib import Path
from urllib.request import urlretrieve

import numpy as np
from pyproj import CRS
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import Window
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
import shapely

crs = CRS.from_epsg(25833)
extent = [-666159.0, 6178074.0, 1275452.0, 9531579.0]

data_dir = Path("data", "gebco")
data_dir.mkdir(parents=True, exist_ok=True)

gebco_theme = "sub_ice_topography_bathymetry"
gebco_files = [
    data_dir / "gebco_2025_sub_ice_n90.0_s0.0_w-90.0_e0.0.tif",
    data_dir / "gebco_2025_sub_ice_n90.0_s0.0_w0.0_e90.0.tif",
]

merge_file = data_dir / "1_merge.tif"
crop_file = data_dir / "2_crop.tif"
warp_file = data_dir / "3_warp.tif"
mask_file = data_dir / "4_mask.tif"
change_file = data_dir / "5_change.tif"
cog_file = data_dir / "6_cog.tif"

## Last ned Gebco-filer

Last ned de relevante filene fra Gebco.

*TODO: Bruke extent til å definere hvilke filer man skal laste ned.*

In [73]:
for gebco_file in gebco_files:
    if gebco_file.exists():
        print(f"File {gebco_file.name} already exists")
        continue

    print(f"Downloading {gebco_file.name}")
    gebco_url = f"https://dap.ceda.ac.uk/bodc/gebco/global/gebco_2025/{gebco_theme}/geotiff/{gebco_file.name}?download=1"
    
    urlretrieve(gebco_url, gebco_file)

File gebco_2025_sub_ice_n90.0_s0.0_w-90.0_e0.0.tif already exists
File gebco_2025_sub_ice_n90.0_s0.0_w0.0_e90.0.tif already exists


## Lagre *bbox* som GeoJSON

Lagre *bbox*, i UTM 33, som GeoJSON. Kan brukes til å validere at resultatet har korrekt utstrekning i f.eks. QGIS.

In [74]:
bbox = shapely.box(*extent)

with data_dir.joinpath("bbox.json").open("w") as fp:
    json.dump({
        "crs": {
            "properties": {
                "name": crs.to_string(),
            },
            "type": "name",
        },
        "geometry": shapely.geometry.mapping(bbox),
        "properties": {},
        "type": "Feature",
    }, fp, indent=4)

## Slå sammen Gebco-filer

For å få sømløse koblinger mellom filene, før de reprojiseres, kan vi slå de sammen først.

In [76]:
img, transform = merge(gebco_files)

with rasterio.open(gebco_files[0]) as src:
    meta: dict = src.meta.copy()
    meta.update({
        "driver": "GTiff",
        "height": img.shape[1],
        "width": img.shape[2],
        "transform": transform,
    })

with rasterio.open(merge_file, "w", **meta) as dst:
    dst.write(img)

## Beskjær bildet

For å gjøre senere prosessering raskere, kan vi beskjære bildet basert på piksler.

Ettersom reprojisering "skviser" bildene jo lenger nord man kommer, må vi ta med en god del mer informasjon for å kunne få til en "firkantet" maske.

*TODO: Regne ut pikselverdier basert på utstrekning.*

In [80]:
with rasterio.open(merge_file) as src:
    window = Window(8_000, 0, 32_000, 9_000)
    transform = src.window_transform(window)
    profile: dict = src.profile.copy()
    profile.update({
        "height": window.height,
        "transform": transform,
        "width": window.width,
    })
    img = src.read(window=window)
    with rasterio.open(crop_file, "w", **profile) as dst:
        dst.write(img)

## Reprojiser bildet

Reprojiser til UTM 33.

In [84]:
with rasterio.open(crop_file) as src:
    transform, width, height = calculate_default_transform(src.crs, crs, src.width, src.height, *src.bounds)
    meta: dict = src.meta.copy()
    meta.update({
        "crs": crs,
        "transform": transform,
        "width": width,
        "height": height
    })
    with rasterio.open(warp_file, "w", **meta) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=crs,
                resampling=Resampling.nearest,
            )

## Masker bildet til utstrekning

Bruk faktisk utstrekning til å maskere bildet.

In [86]:
shapes = [shapely.box(*extent)]

with rasterio.open(warp_file) as src:
    img, transform = mask(src, shapes, crop=True)
    meta: dict = src.meta.copy()
    meta.update({
        "height": img.shape[1],
        "width": img.shape[2],
        "transform": transform,
    })
    with rasterio.open(mask_file, "w", **meta) as dst:
        dst.write(img)

## Endre verdier i bildet

Ignorer elevasjonsverdiene over havnivået med å endre aller verdier som er høyere enn `0` til `1`.

In [91]:
with rasterio.open(mask_file) as src:
    meta: dict = src.meta.copy()
    img = src.read()
    np.place(img, img > 0, 1)
    with rasterio.open(change_file, "w", **meta) as dst:
        dst.write(img)

## Gjør bildet om til en COG!

In [92]:
cog_translate(change_file, cog_file, cog_profiles.get("lzw"))

Reading input: data\gebco\5_change.tif



Adding overviews...
Updating dataset tags...
Writing output to: data\gebco\6_cog.tif
